In [1]:
import tensorflow as tf
tf.enable_eager_execution()

/home/fmsnew/miniconda3/envs/tf_nasbench/lib/python3.6/site-packages/tensorflow/python/framework/dtypes.py:523: FutureWarning: Passing (type, 1) or '1type' as a synonym of type is deprecated; in a future version of numpy, it will be understood as (type, (1,)) / '(1,)type'.
  _np_qint8 = np.dtype([("qint8", np.int8, 1)])
/home/fmsnew/miniconda3/envs/tf_nasbench/lib/python3.6/site-packages/tensorflow/python/framework/dtypes.py:524: FutureWarning: Passing (type, 1) or '1type' as a synonym of type is deprecated; in a future version of numpy, it will be understood as (type, (1,)) / '(1,)type'.
  _np_quint8 = np.dtype([("quint8", np.uint8, 1)])
/home/fmsnew/miniconda3/envs/tf_nasbench/lib/python3.6/site-packages/tensorflow/python/framework/dtypes.py:525: FutureWarning: Passing (type, 1) or '1type' as a synonym of type is deprecated; in a future version of numpy, it will be understood as (type, (1,)) / '(1,)type'.
  _np_qint16 = np.dtype([("qint16", np.int16, 1)])
/home/fmsnew/miniconda3/envs

In [2]:
from __future__ import absolute_import
from __future__ import division
from __future__ import print_function

import time

from nasbench.lib import cifar
from nasbench.lib import model_builder
from nasbench.lib import training_time
import numpy as np


from nasbench.lib import evaluate
from nasbench.lib import model_spec
from nasbench.lib import config as _config

from nasbench.scripts.run_evaluation import NumpyEncoder

import json

RESULTS_FILE = 'results.json'


In [3]:
_config

<module 'nasbench.lib.config' from '/home/fmsnew/misc/nasbench/nasbench/lib/config.py'>

In [4]:
import os
os.environ['AUTOGRAPH_VERBOSITY'] = '10'

In [5]:
config = _config.build_config()
config['use_tpu'] = False
config['train_data_files'] = ['train_1.tfrecords', 'train_2.tfrecords', 'train_3.tfrecords', 'train_4.tfrecords']
config['valid_data_file'] = ['validation.tfrecords']
config['test_data_file'] = ['test.tfrecords']
config['sample_data_file'] = ['sample.tfrecords']
config['train_epochs'] = 36

config['use_KD'] = True
config['imitation_lmb'] = 0.7
config['temperature'] = 20.0

for files_group in ['train_data_files', 'valid_data_file', 'test_data_file', 'sample_data_file']:
    for i in range(len(config[files_group])):
        config[files_group][i] = config[files_group][i].replace('.', '_KD.')

matrix=[[0, 1, 1, 1, 0, 1, 0],    # input layer
      [0, 0, 0, 0, 0, 0, 1],    # 1x1 conv
      [0, 0, 0, 0, 0, 0, 1],    # 3x3 conv
      [0, 0, 0, 0, 1, 0, 0],    # 5x5 conv (replaced by two 3x3's)
      [0, 0, 0, 0, 0, 0, 1],    # 5x5 conv (replaced by two 3x3's)
      [0, 0, 0, 0, 0, 0, 1],    # 3x3 max-pool
      [0, 0, 0, 0, 0, 0, 0]]   # output layer
# Operations at the vertices of the module, matches order of matrix
labels=['input', 'conv1x1-bn-relu', 'conv3x3-bn-relu', 'conv3x3-bn-relu', 'conv3x3-bn-relu', 'maxpool3x3', 'output']


matrix = np.array(matrix)
labels = np.array(labels)

spec = model_spec.ModelSpec(matrix, labels)
model_dir = '../my_model_dir_kd'

In [6]:
from nasbench.lib.evaluate import _TrainAndEvaluator

In [7]:
evaluator = _TrainAndEvaluator(spec, config, model_dir)

INFO:tensorflow:Using config: {'_model_dir': '../my_model_dir_kd', '_tf_random_seed': None, '_save_summary_steps': 100, '_save_checkpoints_steps': None, '_save_checkpoints_secs': 1073741824, '_session_config': allow_soft_placement: true
graph_options {
  rewrite_options {
    meta_optimizer_iterations: ONE
  }
}
, '_keep_checkpoint_max': 3, '_keep_checkpoint_every_n_hours': 10000, '_log_step_count_steps': None, '_train_distribute': None, '_device_fn': None, '_protocol': None, '_eval_distribute': None, '_experimental_distribute': None, '_service': None, '_cluster_spec': <tensorflow.python.training.server_lib.ClusterSpec object at 0x7f9e756ac780>, '_task_type': 'worker', '_task_id': 0, '_global_id_in_cluster': 0, '_master': '', '_evaluation_master': '', '_is_chief': True, '_num_ps_replicas': 0, '_num_worker_replicas': 1, '_tpu_config': TPUConfig(iterations_per_loop=100, num_shards=2, num_cores_per_replica=None, per_host_input_for_training=2, tpu_job_name=None, initial_infeed_sleep_secs=N

In [8]:
#meta = evaluator.run()

In [9]:
input_sample = cifar.CIFARInput('sample', config)

In [10]:
dataset = input_sample.input_fn({'batch_size':100, 'use_KD':config['use_KD']})

Instructions for updating:
Use `tf.data.experimental.map_and_batch(...)`.


In [11]:
dataset = tf.data.TFRecordDataset(config['sample_data_file'])

In [12]:
for element in dataset: 
    #print(element)
    break

In [13]:
element

<tf.Tensor: id=45, shape=(), dtype=string, numpy=b'\n\xce\x18\n9\n\x05label\x120\x12.\n,\x00\x00\x00\x00r\xda\xd9AN\xcf$@\xfe\xc9\xc9\xbewX\xed\xc0Z\xf9t\xc0\x17\xae$\xc1\x7f\xd0\xc2\xc0\xf6\x1e\x95\xc0\xfc\xce\x04@\xcc>8@\n\x90\x18\n\x05image\x12\x86\x18\n\x83\x18\n\x80\x18\xee\xf8\xfd\xfd\xfe\xff\xff\xff\xff\xff\xff\xff\xff\xff\xff\xff\xff\xff\xff\xff\xff\xff\xff\xff\xff\xff\xff\xff\xff\xff\xff\xff\x81\x9e\xf2\xfe\xff\xfc\xfe\xff\xff\xff\xff\xff\xff\xff\xff\xff\xff\xff\xff\xff\xff\xff\xff\xff\xff\xff\xff\xff\xff\xff\xff\xffKX\xb5\xfd\xff\xfc\xfd\xff\xff\xff\xff\xff\xff\xff\xff\xff\xff\xff\xff\xff\xff\xff\xff\xff\xff\xff\xff\xff\xff\xff\xff\xffEEl\xe5\xfe\xfe\xfe\xff\xff\xff\xff\xff\xff\xff\xff\xff\xff\xff\xff\xff\xff\xff\xff\xff\xff\xff\xff\xff\xff\xff\xff\xffDAQ\x9c\xf8\xfe\xff\xff\xff\xff\xff\xff\xff\xff\xff\xff\xff\xff\xff\xff\xff\xff\xff\xff\xff\xff\xff\xff\xff\xff\xff\xffH?Ob\xd0\xfd\xff\xff\xff\xff\xff\xff\xff\xff\xff\xff\xff\xff\xff\xff\xff\xff\xff\xff\xff\xff\xff\xff\xff\xff\

In [15]:
features = tf.parse_single_example(
      element,
      features={
          'image': tf.FixedLenFeature([], tf.string),
          'label': tf.FixedLenFeature([11], tf.float32),
      })

In [16]:
features

{'image': <tf.Tensor: id=50, shape=(), dtype=string, numpy=b'\xee\xf8\xfd\xfd\xfe\xff\xff\xff\xff\xff\xff\xff\xff\xff\xff\xff\xff\xff\xff\xff\xff\xff\xff\xff\xff\xff\xff\xff\xff\xff\xff\xff\x81\x9e\xf2\xfe\xff\xfc\xfe\xff\xff\xff\xff\xff\xff\xff\xff\xff\xff\xff\xff\xff\xff\xff\xff\xff\xff\xff\xff\xff\xff\xff\xff\xffKX\xb5\xfd\xff\xfc\xfd\xff\xff\xff\xff\xff\xff\xff\xff\xff\xff\xff\xff\xff\xff\xff\xff\xff\xff\xff\xff\xff\xff\xff\xff\xffEEl\xe5\xfe\xfe\xfe\xff\xff\xff\xff\xff\xff\xff\xff\xff\xff\xff\xff\xff\xff\xff\xff\xff\xff\xff\xff\xff\xff\xff\xff\xffDAQ\x9c\xf8\xfe\xff\xff\xff\xff\xff\xff\xff\xff\xff\xff\xff\xff\xff\xff\xff\xff\xff\xff\xff\xff\xff\xff\xff\xff\xff\xffH?Ob\xd0\xfd\xff\xff\xff\xff\xff\xff\xff\xff\xff\xff\xff\xff\xff\xff\xff\xff\xff\xff\xff\xff\xff\xff\xff\xff\xff\xffQ@NS\x88\xf2\xfe\xff\xff\xff\xff\xff\xff\xff\xff\xff\xff\xff\xff\xff\xff\xff\xff\xff\xff\xff\xff\xff\xff\xff\xff\xff_APRU\xba\xfe\xff\xff\xff\xff\xff\xff\xff\xff\xff\xff\xff\xff\xff\xff\xff\xff\xff\xff\xff\x

In [20]:
from tensorflow.estimator import ModeKeys

In [21]:
features, labels, input_hooks = (
          evaluator.estimator._get_features_and_labels_from_input_fn(
input_sample.input_fn, ModeKeys.TRAIN))

KeyError: 'use_KD'

In [28]:
features

<tf.Tensor 'IteratorGetNext_1:0' shape=(256, 32, 32, 3) dtype=float32>

In [29]:
labels

<tf.Tensor 'IteratorGetNext_1:1' shape=(256,) dtype=int32>

In [30]:
config

{'train_data_files': ['train_1_KD.tfrecords',
  'train_2_KD.tfrecords',
  'train_3_KD.tfrecords',
  'train_4_KD.tfrecords'],
 'valid_data_file': ['validation_KD.tfrecords'],
 'test_data_file': ['test_KD.tfrecords'],
 'sample_data_file': ['sample_KD.tfrecords'],
 'data_format': 'channels_last',
 'num_labels': 10,
 'module_vertices': 7,
 'max_edges': 9,
 'available_ops': ['conv3x3-bn-relu', 'conv1x1-bn-relu', 'maxpool3x3'],
 'stem_filter_size': 128,
 'num_stacks': 3,
 'num_modules_per_stack': 3,
 'batch_size': 256,
 'train_epochs': 36,
 'train_seconds': 14400.0,
 'learning_rate': 0.1,
 'lr_decay_method': 'COSINE_BY_STEP',
 'momentum': 0.9,
 'weight_decay': 0.0001,
 'max_attempts': 5,
 'intermediate_evaluations': ['0.5'],
 'num_repeats': 3,
 'use_tpu': False,
 'tpu_iterations_per_loop': 100,
 'tpu_num_shards': 2,
 'use_KD': True,
 'imitation_lmb': 0.7,
 'temperature': 20.0}